<a href="https://colab.research.google.com/github/ianellisjones/usn/blob/main/MVT_tracker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
%%shell
# Install Google Chrome Stable
wget -q https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
sudo dpkg -i google-chrome-stable_current_amd64.deb
sudo apt-get -f install -y

# Install Python dependencies
pip install selenium webdriver-manager selenium-stealth beautifulsoup4

Selecting previously unselected package google-chrome-stable.
(Reading database ... 121713 files and directories currently installed.)
Preparing to unpack google-chrome-stable_current_amd64.deb ...
Unpacking google-chrome-stable (142.0.7444.175-1) ...
dpkg: dependency problems prevent configuration of google-chrome-stable:
 google-chrome-stable depends on libatk-bridge2.0-0 (>= 2.5.3); however:
  Package libatk-bridge2.0-0 is not installed.
 google-chrome-stable depends on libatk1.0-0 (>= 2.11.90); however:
  Package libatk1.0-0 is not installed.
 google-chrome-stable depends on libatspi2.0-0 (>= 2.9.90); however:
  Package libatspi2.0-0 is not installed.
 google-chrome-stable depends on libvulkan1; however:
  Package libvulkan1 is not installed.
 google-chrome-stable depends on libxcomposite1 (>= 1:0.4.4-1); however:
  Package libxcomposite1 is not installed.

dpkg: error processing package google-chrome-stable (--install):
 dependency problems - leaving unconfigured
Processing trigge

In [4]:
"""
US NAVY MVT TRACKER (Marine Vessel Traffic)
Target: marinevesseltraffic.com

A specialized scraper that uses Selenium/Stealth to extract AIS data.
Updated to use Google Chrome Stable (Colab "Nuclear" Setup).
"""

import time
import csv
import re
import sys
import os
from typing import List, Dict

# Dependencies
try:
    from selenium import webdriver
    from selenium.webdriver.chrome.options import Options
    from selenium.webdriver.chrome.service import Service
    from webdriver_manager.chrome import ChromeDriverManager
    from selenium_stealth import stealth
    from bs4 import BeautifulSoup
except ImportError:
    print("CRITICAL ERROR: Missing dependencies.")
    print("Please run the Installation Cell first!")
    sys.exit(1)

# --- CONFIGURATION ---
# Target Ships (MVT Specific URLs)
TARGETS = [
    {
        "ship": "USS Gerald R. Ford (CVN 78)",
        "url": "https://www.marinevesseltraffic.com/vessels/USS-Gerald-R.-Ford-(CVN-78)/CURRENT-POSITION/1/338803000"
    }
]

OUTPUT_FILENAME = "mvt_fleet_status.csv"

def fetch_mvt_data(target: Dict) -> Dict:
    """Launches Headless Chrome to fetch MVT data."""
    print(f"[*] Tracking: {target['ship']}...")

    chrome_options = Options()
    # Use the new headless mode for better stability
    chrome_options.add_argument("--headless=new")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument("--window-size=1920,1080")

    # Stealth flags
    chrome_options.add_argument("--disable-blink-features=AutomationControlled")
    chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
    chrome_options.add_experimental_option('useAutomationExtension', False)

    # CRITICAL: Explicitly point to the Google Chrome binary
    chrome_options.binary_location = "/usr/bin/google-chrome"

    driver = None
    result = {
        "Ship": target['ship'],
        "Latitude": "Unknown",
        "Longitude": "Unknown",
        "Last Report": "Unknown",
        "Status": "Unknown"
    }

    try:
        # Install matching driver
        service = Service(ChromeDriverManager().install())
        driver = webdriver.Chrome(service=service, options=chrome_options)

        # Activate Stealth
        stealth(driver,
            languages=["en-US", "en"],
            vendor="Google Inc.",
            platform="Win32",
            webgl_vendor="Intel Inc.",
            renderer="Intel Iris OpenGL Engine",
            fix_hairline=True,
        )

        driver.get(target['url'])

        # Wait for map/data load (MVT is heavy on JS)
        print("    > Waiting 15s for map data...")
        time.sleep(15)

        # --- SCRAPING STRATEGY ---
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        text_content = soup.get_text(" ", strip=True)

        # 1. Coordinate Extraction (Latitude / Longitude: 36.96° / -76.30°)
        # The regex looks for the specific pattern MVT uses
        coord_match = re.search(r'Latitude\s*/\s*Longitude:?\s*([0-9.-]+)°\s*/\s*([0-9.-]+)°', text_content)

        if coord_match:
            result["Latitude"] = coord_match.group(1)
            result["Longitude"] = coord_match.group(2)
            print(f"    > Found Coordinates: {result['Latitude']}, {result['Longitude']}")
        else:
             print("    ! Coordinates not found (Ship might be out of range/AIS off).")

        # 2. Last Report Time
        time_match = re.search(r'Position received:?\s*(.*?)(?=\s*Area|$)', text_content)
        if time_match:
            result["Last Report"] = time_match.group(1)

        # 3. Status
        status_match = re.search(r'Navigational Status:?\s*(\w+)', text_content)
        if status_match:
            result["Status"] = status_match.group(1)

    except Exception as e:
        print(f"    [!] Error: {e}")
        # If binary missing error persists, print help
        if "binary" in str(e).lower():
            print("    > TIP: Did you run the 'apt-get install google-chrome-stable' cell?")

    finally:
        if driver:
            driver.quit()

    return result

def main():
    print(f"{'='*90}")
    print(f"US NAVY MVT TRACKER (MarineVesselTraffic)")
    print(f"{'='*90}\n")

    fleet_data = []

    for target in TARGETS:
        data = fetch_mvt_data(target)
        fleet_data.append(data)
        print(f"    > Data Captured: {data}\n")

    # Save to CSV
    try:
        with open(OUTPUT_FILENAME, 'w', newline='', encoding='utf-8') as f:
            fieldnames = ["Ship", "Latitude", "Longitude", "Last Report", "Status"]
            writer = csv.DictWriter(f, fieldnames=fieldnames)
            writer.writeheader()
            writer.writerows(fleet_data)
        print(f"\n[+] Report saved to '{OUTPUT_FILENAME}'")
    except PermissionError:
        print(f"\n[!] ERROR: Could not write to CSV. Is the file open?")

if __name__ == "__main__":
    main()

US NAVY MVT TRACKER (MarineVesselTraffic)

[*] Tracking: USS Gerald R. Ford (CVN 78)...
    > Waiting 15s for map data...
    ! Coordinates not found (Ship might be out of range/AIS off).
    > Data Captured: {'Ship': 'USS Gerald R. Ford (CVN 78)', 'Latitude': 'Unknown', 'Longitude': 'Unknown', 'Last Report': 'Unknown', 'Status': 'Unknown'}


[+] Report saved to 'mvt_fleet_status.csv'
